In [1]:
spark

# 하둡에서 데이터 불러오기

In [61]:
df = spark.read.csv("hdfs:///electricity/*.csv", encoding='utf-8', header=True)

In [62]:
df.printSchema()

root
 |-- 일시: string (nullable = true)
 |-- 공급능력(MW): string (nullable = true)
 |-- 현재부하(MW): string (nullable = true)
 |-- 공급예비력(MW): string (nullable = true)
 |-- 공급예비율(%): string (nullable = true)



# 데이터 전처리 과정
- 자료형 변경
- 일시 컬럼 분리
- 년, 월, 일 컬럼 생성
- 컬럼 순서 재배치

In [63]:
df = df.withColumn("공급능력(MW)", col("공급능력(MW)").cast("integer")) \
       .withColumn("현재부하(MW)", col("현재부하(MW)").cast("integer")) \
       .withColumn("공급예비력(MW)", col("공급예비력(MW)").cast("integer")) \
       .withColumn("공급예비율(%)", col("공급예비율(%)").cast("double"))

In [64]:
df.printSchema()

root
 |-- 일시: string (nullable = true)
 |-- 공급능력(MW): integer (nullable = true)
 |-- 현재부하(MW): integer (nullable = true)
 |-- 공급예비력(MW): integer (nullable = true)
 |-- 공급예비율(%): double (nullable = true)



In [65]:
df.count()

313191

In [66]:
df.show()

+----------------+------------+------------+--------------+-------------+
|            일시|공급능력(MW)|현재부하(MW)|공급예비력(MW)|공급예비율(%)|
+----------------+------------+------------+--------------+-------------+
|2022-08-23 23:55|      101577|       66527|         35046|         52.7|
|2022-08-23 23:50|      101578|       67053|         34520|         51.5|
|2022-08-23 23:45|      101573|       67623|         33946|         50.2|
|2022-08-23 23:40|      101576|       67904|         33668|         49.6|
|2022-08-23 23:35|      101564|       68725|         32835|         47.8|
|2022-08-23 23:30|      101569|       68845|         32719|         47.5|
|2022-08-23 23:25|      101582|       69010|         32567|         47.2|
|2022-08-23 23:20|      101586|       69327|         32255|         46.5|
|2022-08-23 23:15|      101594|       69907|         31682|         45.3|
|2022-08-23 23:10|      101612|       69922|         31686|         45.3|
|2022-08-23 23:05|      101713|       69758|         31951

In [67]:
# 일시 컬럼 분리하기
df.withColumn("날짜", fn.split("일시", " ")[0]).withColumn("시간", fn.split("일시", " ")[1]).show()

+----------------+------------+------------+--------------+-------------+----------+-----+
|            일시|공급능력(MW)|현재부하(MW)|공급예비력(MW)|공급예비율(%)|      날짜| 시간|
+----------------+------------+------------+--------------+-------------+----------+-----+
|2022-08-23 23:55|      101577|       66527|         35046|         52.7|2022-08-23|23:55|
|2022-08-23 23:50|      101578|       67053|         34520|         51.5|2022-08-23|23:50|
|2022-08-23 23:45|      101573|       67623|         33946|         50.2|2022-08-23|23:45|
|2022-08-23 23:40|      101576|       67904|         33668|         49.6|2022-08-23|23:40|
|2022-08-23 23:35|      101564|       68725|         32835|         47.8|2022-08-23|23:35|
|2022-08-23 23:30|      101569|       68845|         32719|         47.5|2022-08-23|23:30|
|2022-08-23 23:25|      101582|       69010|         32567|         47.2|2022-08-23|23:25|
|2022-08-23 23:20|      101586|       69327|         32255|         46.5|2022-08-23|23:20|
|2022-08-23 23:15|     

In [68]:
df = df.withColumn("날짜", fn.split("일시", " ")[0]).withColumn("시간", fn.split("일시", " ")[1])\
        .withColumn("연도", fn.split("날짜", "-")[0]).withColumn("월", fn.split("날짜", "-")[1]).withColumn("일", fn.split("날짜", "-")[2])\
        .withColumn("일시", col("일시").cast("timestamp")) \
        .drop("날짜")
df.show()

+-------------------+------------+------------+--------------+-------------+-----+----+---+---+
|               일시|공급능력(MW)|현재부하(MW)|공급예비력(MW)|공급예비율(%)| 시간|연도| 월| 일|
+-------------------+------------+------------+--------------+-------------+-----+----+---+---+
|2022-08-23 23:55:00|      101577|       66527|         35046|         52.7|23:55|2022| 08| 23|
|2022-08-23 23:50:00|      101578|       67053|         34520|         51.5|23:50|2022| 08| 23|
|2022-08-23 23:45:00|      101573|       67623|         33946|         50.2|23:45|2022| 08| 23|
|2022-08-23 23:40:00|      101576|       67904|         33668|         49.6|23:40|2022| 08| 23|
|2022-08-23 23:35:00|      101564|       68725|         32835|         47.8|23:35|2022| 08| 23|
|2022-08-23 23:30:00|      101569|       68845|         32719|         47.5|23:30|2022| 08| 23|
|2022-08-23 23:25:00|      101582|       69010|         32567|         47.2|23:25|2022| 08| 23|
|2022-08-23 23:20:00|      101586|       69327|         32255|    

In [69]:
new_column_order = ["일시", "연도", "월", "일", "시간", "공급능력(MW)", "현재부하(MW)", "공급예비력(MW)", "공급예비율(%)"]

In [70]:
df = df.select(new_column_order).orderBy(col("일시"))

df.show()

+-------------------+----+---+---+-----+------------+------------+--------------+-------------+
|               일시|연도| 월| 일| 시간|공급능력(MW)|현재부하(MW)|공급예비력(MW)|공급예비율(%)|
+-------------------+----+---+---+-----+------------+------------+--------------+-------------+
|2021-07-01 00:00:00|2021| 07| 01|00:00|       89264|       61164|         28100|         45.9|
|2021-07-01 00:05:00|2021| 07| 01|00:05|       90856|       61067|         29789|         48.8|
|2021-07-01 00:10:00|2021| 07| 01|00:10|       90847|       61119|         29728|         48.6|
|2021-07-01 00:15:00|2021| 07| 01|00:15|       90833|       60640|         30193|         49.8|
|2021-07-01 00:20:00|2021| 07| 01|00:20|       90804|       60004|         30801|         51.3|
|2021-07-01 00:25:00|2021| 07| 01|00:25|       90796|       59785|         31011|         51.9|
|2021-07-01 00:30:00|2021| 07| 01|00:30|       90794|       59271|         31523|         53.2|
|2021-07-01 00:35:00|2021| 07| 01|00:35|       90795|       59572|

In [72]:
df.groupby("연도", "월", "일").count().filter(col("count") < 288).count()

299

288(12*24) 미만인 애들이 299 일 이란 뜻(null은 count 에 제외되는 코드)

In [73]:
df.show()

+-------------------+----+---+---+-----+------------+------------+--------------+-------------+
|               일시|연도| 월| 일| 시간|공급능력(MW)|현재부하(MW)|공급예비력(MW)|공급예비율(%)|
+-------------------+----+---+---+-----+------------+------------+--------------+-------------+
|2021-07-01 00:00:00|2021| 07| 01|00:00|       89264|       61164|         28100|         45.9|
|2021-07-01 00:05:00|2021| 07| 01|00:05|       90856|       61067|         29789|         48.8|
|2021-07-01 00:10:00|2021| 07| 01|00:10|       90847|       61119|         29728|         48.6|
|2021-07-01 00:15:00|2021| 07| 01|00:15|       90833|       60640|         30193|         49.8|
|2021-07-01 00:20:00|2021| 07| 01|00:20|       90804|       60004|         30801|         51.3|
|2021-07-01 00:25:00|2021| 07| 01|00:25|       90796|       59785|         31011|         51.9|
|2021-07-01 00:30:00|2021| 07| 01|00:30|       90794|       59271|         31523|         53.2|
|2021-07-01 00:35:00|2021| 07| 01|00:35|       90795|       59572|

In [74]:
df.printSchema()

root
 |-- 일시: timestamp (nullable = true)
 |-- 연도: string (nullable = true)
 |-- 월: string (nullable = true)
 |-- 일: string (nullable = true)
 |-- 시간: string (nullable = true)
 |-- 공급능력(MW): integer (nullable = true)
 |-- 현재부하(MW): integer (nullable = true)
 |-- 공급예비력(MW): integer (nullable = true)
 |-- 공급예비율(%): double (nullable = true)



# 데이터 분석

## (3년 전체)시간별 평균 공급능력, 현재부하, 공급예비력, 공급예비율 확인

In [85]:
df.groupby(F.hour("일시")) \
    .agg(F.avg("공급능력(MW)").alias("공급능력(MW)"), \
         F.avg("현재부하(MW)").alias("현재부하(MW)"), \
         F.avg("공급예비력(MW)").alias("공급예비력(MW)"), \
         F.avg("공급예비율(%)").alias("공급예비율(%)")) \
    .show()

+----------+-----------------+------------------+------------------+------------------+
|hour(일시)|     공급능력(MW)|      현재부하(MW)|    공급예비력(MW)|     공급예비율(%)|
+----------+-----------------+------------------+------------------+------------------+
|        12| 92095.2400978743|63992.012846000915|26979.334607738187| 43.45872457562317|
|        22|91384.63570604079| 65710.11727587534| 25672.77298961139|39.235244324740286|
|         1|90715.26374799786| 58888.56097933033|31552.231790099915| 53.58424223934101|
|        13| 92180.8158136684| 65899.88844621513|25537.246245786086|40.486599754826855|
|         6|91117.77611484422| 60731.60148136836|29792.752596212584| 49.13225412339647|
|        16|91761.02100099639| 70315.28527630873| 21235.62527784165| 31.63160113436039|
|         3|91009.15615936498| 57189.95802167608|32878.387803388796|56.837124103190426|
|        20|91559.72092130518| 69486.59378119002|22070.851362763915|32.155163147792706|
|         5|91109.53648854962| 58446.67946564886| 31

### 컬럼별 내림차순으로 비교하기 위해 아래 코드 작성

In [87]:
df.groupby(hour("일시")) \
    .agg(avg("공급능력(MW)").alias("평균 공급능력(MW)")) \
    .orderBy("평균 공급능력(MW)", ascending=False).show()

df.groupby(hour("일시")) \
    .agg(avg("현재부하(MW)").alias("평균 현재부하(MW)")) \
    .orderBy("평균 현재부하(MW)", ascending=False).show()

df.groupby(hour("일시")) \
    .agg(avg("공급예비력(MW)").alias("평균 공급예비력(MW)")) \
    .orderBy("평균 공급예비력(MW)", ascending=False).show()

df.groupby(hour("일시")) \
    .agg(avg("공급예비율(%)").alias("평균 공급예비율(%)")) \
    .orderBy("평균 공급예비율(%)", ascending=False).show()

+----------+-----------------+
|hour(일시)|평균 공급능력(MW)|
+----------+-----------------+
|        13| 92180.8158136684|
|        14|92172.10861394493|
|        10| 92109.0996329714|
|        12| 92095.2400978743|
|        15|92081.45476172205|
|        11|92068.48699709347|
|         9|91825.37457044673|
|        16|91761.02100099639|
|        20|91559.72092130518|
|        21|91515.44157441574|
|        17|91507.15084784778|
|        19|  91505.682065843|
|        18|91469.79378834355|
|         8|91399.63794683776|
|        22|91384.63570604079|
|         7| 91270.3725565058|
|        23|91202.15947185326|
|         6|91117.77611484422|
|         5|91109.53648854962|
|         4|91061.89072999389|
+----------+-----------------+
only showing top 20 rows



+----------+------------------+
|hour(일시)| 평균 현재부하(MW)|
+----------+------------------+
|        18| 71443.07231595092|
|        17| 71081.13389089235|
|        19| 71030.50809607859|
|        16| 70315.28527630873|
|        20| 69486.59378119002|
|        15| 68477.48023942906|
|         9| 67694.42550591829|
|        21| 67299.07095633456|
|        10| 67265.52278635876|
|        14| 67233.52274296233|
|         8|  66923.7990375802|
|        11| 66055.13706593239|
|        13| 65899.88844621513|
|        22| 65710.11727587534|
|        23| 64527.28818785579|
|        12|63992.012846000915|
|         7| 63692.31307269395|
|         0| 60937.12549557792|
|         6| 60731.60148136836|
|         1| 58888.56097933033|
+----------+------------------+
only showing top 20 rows



+----------+-------------------+
|hour(일시)|평균 공급예비력(MW)|
+----------+-------------------+
|         3| 32878.387803388796|
|         4| 32685.944486866218|
|         2| 32443.459649122808|
|         5|  31791.09755725191|
|         1| 31552.231790099915|
|         6| 29792.752596212584|
|         0| 29778.038045135712|
|         7| 27317.382864996947|
|        12| 26979.334607738187|
|        23| 26673.617884250474|
|        22|  25672.77298961139|
|        13| 25537.246245786086|
|        11|   25483.2218142879|
|        10| 24484.268160269155|
|        14| 24376.868067807012|
|         8|  24332.94744882371|
|        21| 24214.032518450185|
|         9| 23855.219549446352|
|        15| 23146.656588135982|
|        20| 22070.851362763915|
+----------+-------------------+
only showing top 20 rows



+----------+------------------+
|hour(일시)|평균 공급예비율(%)|
+----------+------------------+
|         3|56.837124103190334|
|         4|56.284124923640746|
|         2| 55.77554538520213|
|         5|  53.9494198473284|
|         1| 53.58424223934097|
|         6|49.132254123396365|
|         0|49.062511436413516|
|         7| 43.71854001221749|
|        12| 43.45872457562318|
|        23| 41.54021979759649|
|        11| 40.55101728621685|
|        13| 40.48659975482687|
|        22|39.235244324740286|
|        10|38.460360911454394|
|         8| 38.10807363275285|
|        14|38.058648462069485|
|         9|37.321664757541036|
|        21| 36.24681734317344|
|        15| 35.51906223620601|
|        20|32.155163147792685|
+----------+------------------+
only showing top 20 rows



## (3년 전체)월별 평균 공급능력, 현재부하, 공급예비력, 공급예비율 확인
- 컬럼별 내림차순으로 비교하기 위해 아래 코드 작성

In [88]:
df.groupby("월") \
    .agg(avg("공급능력(MW)").alias("평균 공급능력(MW)")) \
    .orderBy("평균 공급능력(MW)", ascending=False).show()

df.groupby("월") \
    .agg(avg("현재부하(MW)").alias("평균 현재부하(MW)")) \
    .orderBy("평균 현재부하(MW)", ascending=False).show()

df.groupby("월") \
    .agg(avg("공급예비력(MW)").alias("평균 공급예비력(MW)")) \
    .orderBy("평균 공급예비력(MW)", ascending=False).show()

df.groupby("월") \
    .agg(avg("공급예비율(%)").alias("평균 공급예비율(%)")) \
    .orderBy("평균 공급예비율(%)", ascending=False).show()

+---+------------------+
| 월| 평균 공급능력(MW)|
+---+------------------+
| 01|102428.71444619187|
| 12|100377.89441555506|
| 08| 99174.58000373762|
| 02| 98758.07024505993|
| 07| 97217.76065351603|
| 09| 90597.02166357738|
| 03|  88113.0791077383|
| 11| 87128.73704377442|
| 06| 85964.95490675041|
| 04|  83506.7800850931|
| 10| 82410.47825761545|
| 05|  81971.1839948149|
+---+------------------+



+---+------------------+
| 월| 평균 현재부하(MW)|
+---+------------------+
| 01| 71346.13794652886|
| 07| 71018.66232495106|
| 12| 71005.40342150877|
| 08| 70129.46940758737|
| 02| 68216.08292762755|
| 11| 63322.72481325231|
| 03|63197.219539304744|
| 06|62824.726067232754|
| 09|62695.938175780044|
| 10| 59012.06649951351|
| 04| 58425.76993637535|
| 05| 58163.69327843227|
+---+------------------+



+---+-------------------+
| 월|평균 공급예비력(MW)|
+---+-------------------+
| 01| 30674.467535385305|
| 02|  30149.92730024956|
| 12|  29012.40865928103|
| 08| 28700.451354886936|
| 09| 27520.109669447018|
| 07| 26032.462204487278|
| 04| 24604.168351614037|
| 03| 24452.156030815222|
| 11| 23414.024112706582|
| 05| 23318.043958976705|
| 10| 22990.680487987425|
| 06| 22839.440735140164|
+---+-------------------+



+---+------------------+
| 월|평균 공급예비율(%)|
+---+------------------+
| 09| 46.10362604263217|
| 02| 45.80152190811287|
| 01| 44.53687935295446|
| 04|    43.15029470315|
| 08|42.827845262567706|
| 12| 42.25618735398297|
| 05|  41.0881581455641|
| 10| 40.04641493900153|
| 03| 39.52691732781418|
| 07| 38.74726321337142|
| 11|37.899674884855024|
| 06| 37.86011011593194|
+---+------------------+



24/07/04 08:18:25 ERROR TaskSchedulerImpl: Lost executor 3 on 192.168.0.212: worker lost
24/07/04 08:18:25 ERROR TaskSchedulerImpl: Lost executor 4 on 192.168.0.212: worker lost


## 연도별 평균 공급능력, 현재부하, 공급예비력, 공급예비율 확인
- 1년 전체가 있는 년도는 2년치 뿐이라 이는 시각화에 사용 X

In [94]:
df.filter((col("연도")==2021) | (col("연도")==2022)).groupby("연도") \
    .agg(avg("공급능력(MW)").alias("평균 공급능력(MW)")) \
    .orderBy("평균 공급능력(MW)", ascending=False).show()

df.filter((col("연도")==2021) | (col("연도")==2022)).groupby("연도") \
    .agg(avg("현재부하(MW)").alias("평균 현재부하(MW)")) \
    .orderBy("평균 현재부하(MW)", ascending=False).show()

df.filter((col("연도")==2021) | (col("연도")==2022)).groupby("연도") \
    .agg(avg("공급예비력(MW)").alias("평균 공급예비력(MW)")) \
    .orderBy("평균 공급예비력(MW)", ascending=False).show()

df.filter((col("연도")==2021) | (col("연도")==2022)).groupby("연도") \
    .agg(avg("공급예비율(%)").alias("평균 공급예비율(%)")) \
    .orderBy("평균 공급예비율(%)", ascending=False).show()

+----+-----------------+
|연도|평균 공급능력(MW)|
+----+-----------------+
|2022|91295.64099080872|
|2021|91051.95866328193|
+----+-----------------+



+----+-----------------+
|연도|평균 현재부하(MW)|
+----+-----------------+
|2021|65999.83904823249|
|2022| 65955.3475538085|
+----+-----------------+



+----+-------------------+
|연도|평균 공급예비력(MW)|
+----+-------------------+
|2022|  25093.66491319344|
|2021| 25053.147539120222|
+----+-------------------+



+----+------------------+
|연도|평균 공급예비율(%)|
+----+------------------+
|2021| 39.63921304891461|
|2022|   39.430789833709|
+----+------------------+

